In [ ]:
# Requirements for this notebook to run: pandas, geopandas, shapely, pyproj
import time
import json

import pandas
import geopandas
from shapely.geometry import shape
import contextily as ctx

from eocalc.context import Pollutant
from eocalc.methods.base import DateRange
from eocalc.methods.temis import TEMISTropomiMonthlyMeanNOxEmissionCalculator

In [ ]:
with (open("data/regions/germany.geo.json", 'r')) as geojson_file:
    region = shape(json.load(geojson_file)["geometry"])

results = TEMISTropomiMonthlyMeanNOxEmissionCalculator().run(
            region, DateRange(start='2020-01-01', end='2020-03-15'), Pollutant.NOx)

In [ ]:
results

In [ ]:
results[TEMISTropomiMonthlyMeanNOxEmissionCalculator.TOTAL_EMISSIONS_KEY]

In [ ]:
gridded_result = results[TEMISTropomiMonthlyMeanNOxEmissionCalculator.GRIDDED_EMISSIONS_KEY]
gridded_result.plot("total", figsize=(20, 20), alpha=0.3, edgecolor='k')

In [ ]:
ctx.add_basemap(gridded_result.to_crs(epsg=3857).plot(figsize=(10, 10), alpha=0.3, edgecolor='k'))